## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
del application_df["EIN"]
del application_df["NAME"]

# Also drop STATUS since there only 5 non-active rows
del application_df["STATUS"]

# Surprisingly, the overwhelming about of loans are asking for the same amount of money, $5000,
# which could badly skew the model. So drop that column.
#!!! del application_df["ASK_AMT"]

In [3]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
v_counts = application_df["APPLICATION_TYPE"].value_counts()
v_counts

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = v_counts[v_counts < 50].index.values

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure replacement was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
T9         156
T13         66
Other       54
Name: count, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts to identify and replace with "Other"
v_counts = application_df["CLASSIFICATION"].value_counts()
v_counts

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64

In [7]:
# You may find it helpful to look at CLASSIFICATION value counts >1
v_counts[v_counts > 1]

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: count, dtype: int64

In [8]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = v_counts[v_counts < 100].index.values

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure replacement was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
Other      669
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
Name: count, dtype: int64

In [9]:
# The INCOME_AMT column is categorical but could be numerical, which might be helpful.
application_df["INCOME_MID"] = application_df["INCOME_AMT"].map({
    "0": 0,
    "25000-99999": 62499.5,
    "100000-499999": 299999.5,
    "1M-5M": 3000000,
    "1-9999": 5000,
    "10000-24999": 17499.5,
    "10M-50M": 30000000,
    "5M-10M": 7500000,
    "50M+": 50000000})
del application_df["INCOME_AMT"]
application_df["INCOME_MID"].value_counts()

INCOME_MID
0.0           24388
62499.5        3747
299999.5       3374
3000000.0       955
5000.0          728
17499.5         543
30000000.0      240
7500000.0       185
50000000.0      139
Name: count, dtype: int64

In [10]:
# Replace the ask and income columns with a ratio
application_df["INCOME_ASK_RATIO"] = application_df["INCOME_MID"] / application_df["ASK_AMT"]

In [11]:
# Change the SPECIAL_CONSIDERATIONS column into something usable
application_df["SPECIAL_CONSIDERATIONS"] = application_df["SPECIAL_CONSIDERATIONS"].map({"Y":1, "N":0})

In [12]:
# Convert categorical data to numeric with `pd.get_dummies`
cat_columns = ["APPLICATION_TYPE", "AFFILIATION", "CLASSIFICATION", "USE_CASE", "ORGANIZATION"]
dummied_df = pd.get_dummies(application_df, columns=cat_columns)
dummied_df.head()

,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL,INCOME_MID,INCOME_ASK_RATIO,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T13,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,...,CLASSIFICATION_Other,USE_CASE_CommunityServ,USE_CASE_Heathcare,USE_CASE_Other,USE_CASE_Preservation,USE_CASE_ProductDev,ORGANIZATION_Association,ORGANIZATION_Co-operative,ORGANIZATION_Corporation,ORGANIZATION_Trust
0,0,5000,1,0.0,0.000000,False,True,False,False,False,...,False,False,False,False,False,True,True,False,False,False
1,0,108590,1,5000.0,0.046045,False,False,False,False,True,...,False,False,False,False,True,False,False,True,False,False
2,0,5000,0,0.0,0.000000,False,False,False,False,False,...,False,False,False,False,False,True,True,False,False,False
3,0,6692,1,17499.5,2.614988,False,False,False,False,True,...,False,False,False,False,True,False,False,False,False,True
4,0,142590,1,299999.5,2.103931,False,False,False,False,True,...,False,False,True,False,False,False,False,False,False,True


In [13]:
# Split our preprocessed data into our features and target arrays
X = dummied_df.loc[:, dummied_df.columns != "IS_SUCCESSFUL"]
y = dummied_df["IS_SUCCESSFUL"]

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=8675309)
X.shape, y.shape, X_train.shape, X_test.shape, y_train.shape, y_test.shape

((34299, 42), (34299,), (25724, 42), (8575, 42), (25724,), (8575,))

In [14]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
X_train_scaled.shape, X_train_scaled, X_test_scaled.shape, X_test_scaled

((25724, 42),
 array([[-2.85836535e-02, -3.20858149e-02, -1.41177147e-01, ...,
         -1.20471284e-01, -3.41699834e-02, -1.46383819e+00],
        [-2.85836535e-02, -1.62123331e-04, -1.26153290e-01, ...,
         -1.20471284e-01, -3.41699834e-02,  6.83135615e-01],
        [-2.85836535e-02, -2.73637489e-03,  5.79973794e-01, ...,
         -1.20471284e-01, -3.41699834e-02,  6.83135615e-01],
        ...,
        [-2.85836535e-02, -3.20858149e-02, -1.41177147e-01, ...,
         -1.20471284e-01, -3.41699834e-02, -1.46383819e+00],
        [-2.85836535e-02, -3.20858149e-02, -1.41177147e-01, ...,
         -1.20471284e-01, -3.41699834e-02,  6.83135615e-01],
        [-2.85836535e-02, -3.20858149e-02, -1.41177147e-01, ...,
         -1.20471284e-01, -3.41699834e-02,  6.83135615e-01]]),
 (8575, 42),
 array([[-0.02858365, -0.03208581, -0.14117715, ..., -0.12047128,
         -0.03416998,  0.68313562],
        [-0.02858365, -0.03208581, -0.14117715, ..., -0.12047128,
         -0.03416998, -1.46383819]

## Compile, Train and Evaluate the Model

In [15]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
in_shape = (X.shape[1],)
nn = tf.keras.models.Sequential()

# Input layer
nn.add(tf.keras.Input(shape=in_shape))

units = 1
# First hidden layer
#nn.add(tf.keras.layers.Dense(units=units, activation="elu"))

# Second hidden layer
#nn.add(tf.keras.layers.Dense(units=units, activation="elu"))

#As = ["tanh", "relu", "elu", "selu", "exponential", "gelu", "mish", "relu6", "silu"]
As = ["relu", "elu", "selu", "gelu", "mish"]
us = [40, 30, 20, 10]

for u in us:
    for a in As:
        nn.add(tf.keras.layers.Dense(units=u, activation=a))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 40)                  │           1,720 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 40)                  │           1,640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 40)                  │           1,640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 40)                  │           1,640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 40)                  │           1,640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 30)                  │           1,230 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 30)                  │             930 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 30)                  │             930 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (None, 30)                  │             930 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ (None, 30)                  │             930 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_10 (Dense)                     │ (None, 20)                  │             620 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_11 (Dense)                     │ (None, 20)                  │             420 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_12 (Dense)                     │ (None, 20)                  │             420 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_13 (Dense)                     │ (None, 20)                  │             420 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_14 (Dense)                     │ (None, 20)                  │             420 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_15 (Dense)                     │ (None, 10)                  │             210 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_16 (Dense)                     │ (None, 10)                  │             110 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_17 (Dense)                     │ (None, 10)                  │             110 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_18 (Dense)                     │ (None, 10)                  │             110 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_19 (Dense)                     │ (None, 10)                  │             110 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_20 (Dense)                     │ (None, 1)                   │              11 │
└──────────────────────────────────────┴─────────────────────────────┴──────────────

 Total params: 16,191 (63.25 KB)

 Trainable params: 16,191 (63.25 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
EPOCHS = 50
optimizers = ["adadelta", "adafactor", "adagrad", "adam", "adamw", "adamax", "ftrl", "lion", "nadam", "rmsprop", "sgd"]
results = {}

# Callback to save whole model every 5 epochs
class CustomSaver(tf.keras.callbacks.Callback): 
    def on_epoch_end(self, epoch, logs={}): 
    # Save the model weights every 5 epochs 
        if (epoch + 1) % 5 == 0:
            self.model.save(f'model_epoch_{epoch + 1:03}.h5')
            print(f'Saved model weights at epoch {epoch + 1}')
saver = CustomSaver()

for o in optimizers:




    
    in_shape = (X.shape[1],)
    nn = tf.keras.models.Sequential()
    
    # Input layer
    nn.add(tf.keras.Input(shape=in_shape))
    
    units = 1
    # First hidden layer
    #nn.add(tf.keras.layers.Dense(units=units, activation="elu"))
    
    # Second hidden layer
    #nn.add(tf.keras.layers.Dense(units=units, activation="elu"))
    
    #As = ["tanh", "relu", "elu", "selu", "exponential", "gelu", "mish", "relu6", "silu"]
    As = ["relu", "elu", "selu", "gelu", "mish"]
    us = [40, 30, 20, 10]
    
    for u in us:
        for a in As:
            nn.add(tf.keras.layers.Dense(units=u, activation=a))
    
    # Output layer
    nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))







    # Compile the model
    #nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
    nn.compile(loss="binary_crossentropy", optimizer=o, metrics=["accuracy"])    
    
    # Train the model
    #nn.fit(X_train_scaled, y_train, epochs=EPOCHS, callbacks=[saver])
    nn.fit(X_train_scaled, y_train, epochs=EPOCHS)
    
    model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
    print(f"Optimizer: {o}, Loss: {model_loss}, Accuracy: {model_accuracy}")
    results[o] = model_accuracy

results

Epoch 1/50
804/804 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - accuracy: 0.4679 - loss: 0.6932
Epoch 2/50
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.5364 - loss: 0.6931
Epoch 3/50
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.5621 - loss: 0.6929
Epoch 4/50
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.5566 - loss: 0.6929
Epoch 5/50
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.5461 - loss: 0.6928
Epoch 6/50
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.5373 - loss: 0.6927
Epoch 7/50
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.5436 - loss: 0.6925
Epoch 8/50
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.5330 - loss: 0.6925
Epoch 9/50
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.5314 - loss: 0.6925
Epoch 10/50
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.5301 - loss: 0.6924
Epoch 11/50
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.5317 - loss: 0.6923
Epoch 12/50
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step

In [17]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - 1ms/step - accuracy: 0.5283 - loss: 0.6911
Loss: 0.6910977363586426, Accuracy: 0.5282799005508423


In [18]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_Optimization.h5")